#  Setup stuff: Display options

In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


#  Setup stuff: Connectivity

In [2]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [ ]:

#  DELETE ALL GRAPHS

#  for l_database in my_client.databases():
#     for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
#        l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
#        l_handle.delete()
#  
#  for l_graph in my_client.graphs():
#     print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)
#  
#  print("--")


In [ ]:

#  DELETE ALL DATABASES

#  for l_database in my_client.databases():
#     if (l_database.name != "default"):
#        my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:

#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


In [4]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, 7hJbzBCvDYJxtz881CHHhs38K6V9TWwUB6Feyc2NChDu, 1>


# Create: Vertices/nodes, edges ..

In [ ]:
# import pandas as pd
# 
# print("--")


In [ ]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   #  ], columns = ["airport_code", "airport_name", "LABEL"])
   ], columns = ["id", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


In [ ]:

#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

print(tabulate(df_flights, headers='keys', tablefmt='psql'))


In [ ]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      #  id_column             = "airport_code",
      id_column             = "id",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


In [ ]:

#  Used to test heterogeneous graphs ..

l_query = """
   CREATE ( n: Restaurant { id: 'PAP' } )                         //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.id               = 'SJC'
   AND   m.id               = 'PAP'
   CREATE (n) -[r: ATTRACTION { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result = my_graph.query(l_query))


#  Diagnostics

In [5]:

#  What's in the graph ..

l_result = my_graph.query_unpaginated("""
   MATCH ( n ) 
   RETURN n
   """)
print(tabulate(l_result, headers='keys', tablefmt='psql'))

l_result = my_graph.query_unpaginated("""
   MATCH ( n ) - [ r ] -> ( m )
   RETURN r
   """)
print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/result_set.py:69: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._response = response


+----+---------+-------------+-----------+------------------+------------------+---------------+
|    |   id(n) | labels(n)   | n.LABEL   | n.airport_code   | n.airport_name   | datatype(n)   |
|----+---------+-------------+-----------+------------------+------------------+---------------|
|  0 |       0 | ['Airport'] | Airport   | MKE              | Milwaukee        | node          |
|  1 |       1 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   | node          |
|  2 |       3 | ['Airport'] | Airport   | DEN              | Denver           | node          |
|  3 |       2 | ['Airport'] | Airport   | SJC              | San Jose         | node          |
+----+---------+-------------+-----------+------------------+------------------+---------------+


          0/? [?op/s]

+----+------------------+----------------------+---------+-----------+--------------+--------------+----------+--------------------+------------------------+---------------+
|    |   id(endNode(r)) | labels(endNode(r))   |   id(r) | type(r)   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |   id(startNode(r)) | labels(startNode(r))   | datatype(r)   |
|----+------------------+----------------------+---------+-----------+--------------+--------------+----------+--------------------+------------------------+---------------|
|  0 |                0 | ['Airport']          |       0 | FLIES_TO  |           66 |            1 | FLIES_TO |                  1 | ['Airport']            | edge          |
|  1 |                3 | ['Airport']          |       5 | FLIES_TO  |          886 |            1 | FLIES_TO |                  1 | ['Airport']            | edge          |
|  2 |                1 | ['Airport']          |       2 | FLIES_TO  |         1829 |            1 | FLIES_TO |                  2

In [ ]:

l_result = my_graph.query("""
   MATCH ( n ) - [ r ] -> ( m )
   RETURN n, r, m
   """, contextualize=True)

l_result.view()
